This notebook generates a displacement adjacement matrix for stations.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from functools import reduce
from geopy import Point
from geopy import distance

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
class AnalysisConfig:
    def __init__(self, csv_output_path='', stations_path=''):
        self.csv_output_path = csv_output_path
        self.stations_path = stations_path

In [3]:
STATIONS_DIR = '../data/exports'
SHAPEFILE_DIR = '../data/shapefiles/zipcodes'

config_sf = AnalysisConfig(
    csv_output_path='{}/displacement_adjacency_matrix_san_francisco.csv'.format(
        STATIONS_DIR),
    stations_path='{}/stations_san_francisco.csv'.format(STATIONS_DIR),
)

config_dc = AnalysisConfig(
    csv_output_path='{}/displacement_adjacency_matrix_washington_dc.csv'.format(
        STATIONS_DIR),
    stations_path='{}/stations_washington_dc.csv'.format(STATIONS_DIR),
)

In [4]:
config = config_sf

In [5]:
def get_distance(point1, point2):
    return distance.geodesic(point1, point2).m


def get_point(station_id):
    return Point(stations_df.loc[station_id]['lat'],
                 stations_df.loc[station_id]['lng'])


def get_displacement(station_id1, station_id2):
    return get_distance(get_point(station_id1), get_point(station_id2))

In [6]:
stations_df = pd.read_csv(config.stations_path)
stations_df.set_index('station_id', inplace=True)
stations_df.head()

,lat,lng
station_id,,
31000,38.858970,-77.053239
31001,38.857250,-77.053402
31002,38.856408,-77.049262
31003,38.860856,-77.049425
31004,38.857866,-77.059490


In [7]:
adj_matrix = pd.DataFrame(index=stations_df.index, columns=stations_df.index)
adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
    lambda col_name: get_displacement(row.name, col_name)),
                              axis=1)
adj_matrix.head()

station_id,31000,31001,31002,31003,31004,31005,31006,31007,31009,31010,...,32417,32418,32419,32420,32421,32422,31711,31715,31815,32409
station_id,,,,,,,,,,,,,,,,,,,,,
31000,0.000000,191.524539,447.301979,391.701015,556.248804,686.981398,1017.201538,251.485118,1175.693898,1835.870924,...,15252.678999,13001.395076,12593.347228,14607.824334,15338.929568,14920.276500,11782.670434,9372.700787,6783.096834,7082.510689
31001,191.524539,0.000000,371.352284,528.628446,532.822114,795.265797,1116.028675,197.836270,988.834353,1649.389831,...,15415.512835,13167.015697,12658.221972,14771.477975,15491.456803,15084.959052,11885.478874,9442.464561,6759.183726,7042.124234
31002,447.301979,371.352284,0.000000,493.910366,902.452859,1132.163842,1460.931399,197.134274,902.826468,1534.741354,...,15284.815775,13047.067722,12339.803945,14643.849811,15328.343609,14960.859173,11617.468569,9130.476600,6388.768190,6670.842593
31003,391.701015,528.628446,493.910366,0.000000,934.577020,925.265969,1244.508286,409.762851,1387.097907,2027.386956,...,14888.930848,12642.035406,12219.613513,14245.310127,14962.846309,14559.352366,11392.201206,8996.285647,6509.598518,6834.474138
31004,556.248804,532.822114,902.452859,934.577020,0.000000,491.252076,723.534166,727.126085,1252.719835,1876.120151,...,15676.282460,13415.679222,13149.121568,15028.725451,15794.541979,15337.274557,12320.561494,9927.767046,7291.220751,7569.786339


In [8]:
adj_matrix.to_csv(config.csv_output_path)

In [9]:
# Example for retrieving displacement from SF.
csv_adj_matrix = pd.read_csv(config.csv_output_path).set_index('station_id')
display(csv_adj_matrix.head())

csv_adj_matrix['SF-AA15']['SF-Y14']

,31000,31001,31002,31003,31004,31005,31006,31007,31009,31010,...,32417,32418,32419,32420,32421,32422,31711,31715,31815,32409
station_id,,,,,,,,,,,,,,,,,,,,,
31000,0.000000,191.524539,447.301979,391.701015,556.248804,686.981398,1017.201538,251.485118,1175.693898,1835.870924,...,15252.678999,13001.395076,12593.347228,14607.824334,15338.929568,14920.276500,11782.670434,9372.700787,6783.096834,7082.510689
31001,191.524539,0.000000,371.352284,528.628446,532.822114,795.265797,1116.028675,197.836270,988.834353,1649.389831,...,15415.512835,13167.015697,12658.221972,14771.477975,15491.456803,15084.959052,11885.478874,9442.464561,6759.183726,7042.124234
31002,447.301979,371.352284,0.000000,493.910366,902.452859,1132.163842,1460.931399,197.134274,902.826468,1534.741354,...,15284.815775,13047.067722,12339.803945,14643.849811,15328.343609,14960.859173,11617.468569,9130.476600,6388.768190,6670.842593
31003,391.701015,528.628446,493.910366,0.000000,934.577020,925.265969,1244.508286,409.762851,1387.097907,2027.386956,...,14888.930848,12642.035406,12219.613513,14245.310127,14962.846309,14559.352366,11392.201206,8996.285647,6509.598518,6834.474138
31004,556.248804,532.822114,902.452859,934.577020,0.000000,491.252076,723.534166,727.126085,1252.719835,1876.120151,...,15676.282460,13415.679222,13149.121568,15028.725451,15794.541979,15337.274557,12320.561494,9927.767046,7291.220751,7569.786339


KeyError: 'SF-AA15'